In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
from src.sentiment_analysis_BERT import *

input_df = pd.read_csv('../data/processed/cleaned_vertuo_pop_comments.csv')

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
analyzer = BERTSentimentAnalyzer()
results_df = analyzer.analyze_dataframe(input_df)

Using device: cuda
Loading BERT model...


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully
Analyzing 245 comments...


Analyzing sentiment: 100%|██████████| 16/16 [00:00<00:00, 33.96it/s]


In [4]:
print(
"""
Video Statistics Summary:
--------------------------------------------------
Total Views: 1,149,975
Total Likes: 3,653
Total Dislikes: 278（7%）
Total Comments: 446 (including replies)
--------------------------------------------------
""")


Video Statistics Summary:
--------------------------------------------------
Total Views: 1,149,975
Total Likes: 3,653
Total Dislikes: 278（7%）
Total Comments: 446 (including replies)
--------------------------------------------------



In [5]:
quick_sentiment_summary(results_df)


Sentiment Summary:
--------------------------------------------------
Sentiment       Count  %      Total Likes 
--------------------------------------------------
Negative       :  135 ( 55.1%)       1442 likes
Neutral        :   81 ( 33.1%)        561 likes
Positive       :   29 ( 11.8%)        173 likes
--------------------------------------------------

Overall Summary:
------------------------------
Negative:  55.1% ( 135 comments,   1442 likes)
Neutral:   33.1% (  81 comments,    561 likes)
Positive:  11.8% (  29 comments,    173 likes)
Total:    100.0% ( 245 comments,   2176 likes)

Engagement Insights:
Negative comments received 66.3% of total likes
Positive comments received 8.0% of total likes

Average likes per comment:
Negative: 10.7 likes/comment
Neutral:  6.9 likes/comment
Positive: 6.0 likes/comment

Risk Assessment:
HIGH NEGATIVE SENTIMENT - Requires immediate attention


In [6]:
validator = SentimentValidator(results_df)

In [7]:
validation_report = validator.full_validation_report(save_manual_sample = False)

SENTIMENT ANALYSIS VALIDATION REPORT

Statistical Validation:
------------------------------
Sentiment Distribution:
Negative       :  55.1%
Neutral        :  33.1%
Positive       :  11.8%

Likes by Sentiment:
                 count   sum   mean
sentiment_label                    
Negative           135  1442  10.68
Neutral             81   561   6.93
Positive            29   173   5.97

Statistical Warnings:
Negative comments receiving high engagement - major issue

Confidence Distribution Analysis:
----------------------------------------
Very Low (<0.5):   10 (  4.1%)
Low (0.5-0.7):    56 ( 22.9%)
Medium (0.7-0.9): 101 ( 41.2%)
High (>0.9):       78 ( 31.8%)

Confidence Statistics:
Mean: 0.790
Std:  0.150
Min:  0.409
Max:  0.986


In [8]:
validator.analyze_confidence_distribution()


Confidence Distribution Analysis:
----------------------------------------
Very Low (<0.5):   10 (  4.1%)
Low (0.5-0.7):    56 ( 22.9%)
Medium (0.7-0.9): 101 ( 41.2%)
High (>0.9):       78 ( 31.8%)

Confidence Statistics:
Mean: 0.790
Std:  0.150
Min:  0.409
Max:  0.986


{'statistics': count    245.000000
 mean       0.790453
 std        0.149825
 min        0.409208
 25%        0.668011
 50%        0.829471
 75%        0.916433
 max        0.985745
 Name: sentiment_confidence, dtype: float64,
 'distribution': {'very_low': 10, 'low': 56, 'medium': 101, 'high': 78},
 'warnings': [],
 'low_confidence_samples':                                                   text sentiment_label  \
 87   What's the problem with having more buttons???...         Neutral   
 88   Just got mine out of the box and prepared a co...         Neutral   
 210  Just got mine out of the box but I'm thinking ...        Positive   
 215                               This is so complex !         Neutral   
 167  Thanks to tik tok i got it ! This video is so ...        Negative   
 1    Instructions from me to Nespresso, add a freak...         Neutral   
 56   The puzzles on Hogwarts Legacy were easier to ...         Neutral   
 38   Hmmm followed the descaling instructions to th...  

In [14]:
validation_df = validator.create_low_confidence_validation_sample(save_path='../data/validation/manual_validation_sample.csv')
amended_df = pd.read_csv('../data/validation/edited_val_sample.csv')

Found 66 low confidence predictions (< 0.7)
Low confidence validation sample saved to ../data/validation/manual_validation_sample.csv
Instructions: Fill 'manual_label' column with correct sentiment (Positive/Negative/Neutral)


In [15]:
amended_df

,text,sentiment_label,sentiment_confidence,manual_label,notes
0,What's the problem with having more buttons???...,Neutral,0.409208,Negative,NaN
1,Just got mine out of the box and prepared a co...,Neutral,0.420297,Negative,NaN
2,Just got mine out of the box but I'm thinking ...,Positive,0.432478,Negative,NaN
3,This is so complex !,Neutral,0.442402,Negative,NaN
4,Thanks to tik tok i got it ! This video is so ...,Negative,0.453931,Negative,NaN
...,...,...,...,...,...
61,Can y'all just make a descaling pod?,Neutral,0.668011,Negative,NaN
62,I just get a red light,Neutral,0.676840,Neutral,NaN
63,THIS IS INSANITY,Negative,0.680063,Negative,NaN
64,K… k… kurzgesagd?.. That was unexpected 😁,Neutral,0.689647,Neutral,NaN


In [16]:
final_sentiment_df = validator.apply_manual_corrections(amended_df)

Applying 66 manual corrections...

1.Turn mach...
It’s ...
Successfully applied 62 manual corrections
Corrected predictions: 62
Original BERT predictions: 183


In [18]:
final_sentiment_df

,comment_id,author,author_channel_id,text,like_count,published_at,updated_at,cleaned_text,original_length,cleaned_length,sentiment_label,sentiment_confidence,sentiment_prediction,prob_negative,prob_neutral,prob_positive,manual_correction
0,UgzeXSelTVLuObwtCf14AaABAg,@ktom,UCugDBAll0jwAthUaeZzQGLA,Nice to see Kurzgesagt getting into the coffee...,1,2025-05-22T20:09:06Z,2025-05-22T20:09:06Z,Nice to see Kurzgesagt getting into the coffee...,54,53,Positive,0.955166,2,0.003038,0.041796,0.955166,False
1,UgxKGip-_2KtOR7zUoh4AaABAg,@ramong26,UCQye_QAIcLtyf4hMH0hCNHw,"Instructions from me to Nespresso, add a freak...",0,2025-05-21T15:19:51Z,2025-05-21T15:19:51Z,"Instructions from me to Nespresso, add a freak...",93,93,Negative,0.477912,1,0.260043,0.477912,0.262045,True
2,UgyR2F6VnpfGcS-35lh4AaABAg,@spartanGTA6,UCeAprMuT4efhc5V_UozwbDA,Hey Nespresso. Great product. Got this recentl...,0,2025-05-15T20:48:40Z,2025-05-15T20:48:40Z,Hey Nespresso. Great product. Got this recentl...,190,188,Positive,0.942392,2,0.006181,0.051426,0.942392,False
3,UgzWVTLMraECmNlRQU94AaABAg,@vanessam.v2779,UCOkEKQZR7Tkcqyi-qXH2X9w,wow only 432 steps?! so simple,0,2025-05-14T15:15:13Z,2025-05-14T15:15:13Z,wow only 432 steps?! so simple,31,30,Positive,0.913963,2,0.017165,0.068873,0.913963,False
4,Ugx7Vrvc6XSCYQ9qJCF4AaABAg,@bannermanfamilyfmailly-l8l,UCmuBtP1U_p33MY7cwF6YPYg,This is insanley complex and hasnt worked for ...,1,2025-05-03T18:27:45Z,2025-05-03T18:27:45Z,This is insanley complex and hasnt worked for ...,55,55,Negative,0.904226,0,0.904226,0.085477,0.010297,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,UgyRJ9yeu5n2gm5Nrmd4AaABAg,@peaks.2882,UCllsBJfEmMJZNNfheW67rRg,Why steps need to be this complicated? Not use...,5,2022-12-04T21:58:52Z,2022-12-04T21:58:52Z,Why steps need to be this complicated? Not use...,63,63,Negative,0.917598,0,0.917598,0.075556,0.006846,False
241,UgyNCqf-MrLHfiipfKh4AaABAg,@PB111627,UCvd6DUdGTTIQWOf3W7O8zfw,Do a programming course before attempting this...,124,2022-12-01T22:45:50Z,2022-12-01T22:45:50Z,Do a programming course before attempting this...,55,55,Negative,0.822619,0,0.822619,0.158901,0.018480,False
242,Ugz888TJ2cTz351SnU94AaABAg,@dhedc,UCF7U08w8ozQ8Rz3pDOSJMQg,Very complicated,11,2022-10-03T00:16:27Z,2022-10-03T00:16:27Z,Very complicated,16,16,Negative,0.526147,1,0.393441,0.526147,0.080411,True
243,Ugw9QlYPNhRaPAuPMd14AaABAg,@SouthFacedWindows,UCp2XzdIoSa5chk63hczr0pg,The process of descaling could not be any more...,29,2022-08-09T13:50:18Z,2022-08-09T13:50:18Z,The process of descaling could not be any more...,116,116,Negative,0.704269,0,0.704269,0.274676,0.021055,False


In [19]:
quick_sentiment_summary(final_sentiment_df)


Sentiment Summary:
--------------------------------------------------
Sentiment       Count  %      Total Likes 
--------------------------------------------------
Negative       :  161 ( 65.7%)       1688 likes
Neutral        :   59 ( 24.1%)        338 likes
Positive       :   25 ( 10.2%)        150 likes
--------------------------------------------------

Overall Summary:
------------------------------
Negative:  65.7% ( 161 comments,   1688 likes)
Neutral:   24.1% (  59 comments,    338 likes)
Positive:  10.2% (  25 comments,    150 likes)
Total:    100.0% ( 245 comments,   2176 likes)

Engagement Insights:
Negative comments received 77.6% of total likes
Positive comments received 6.9% of total likes

Average likes per comment:
Negative: 10.5 likes/comment
Neutral:  5.7 likes/comment
Positive: 6.0 likes/comment

Risk Assessment:
HIGH NEGATIVE SENTIMENT - Requires immediate attention


In [20]:
final_sentiment_df.to_csv('../data/exports/final_sentiment_analysis_results.csv', index=False)